In [27]:
# Import necessary libraries

from dotenv import load_dotenv
from openai import OpenAI
import os
import json
import requests
from pypdf import PdfReader
from IPython.display import Markdown
import gradio as gr

In [28]:
# initialize the environment
load_dotenv(override=True)

openai_client = OpenAI()
model_name = "gpt-4o-mini"

# ollama_api_key = os.getenv('OLLAMA_API_KEY')
# ollama_client = OpenAI(api_key=ollama_api_key, base_url="http://localhost:11434/v1")
# model_name = "llama3.2:latest"

# messages = [
#     {"role": "system", "content": "You are a helpful assistant"},
#     {"role": "user", "content": "What is the capital of France?"}
# ]

# response = ollama_client.chat.completions.create(model=model_name, messages=messages, temperature=0.7)

# answer = response.choices[0].message.content
# display(Markdown(answer))

In [ ]:
# Pushover configs

pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = f"https://api.pushover.net/1/messages.json"

In [ ]:
# create a function to send a pushover notification

def push(message):
    print(f"Push {message}")
    payload = {
        "token": pushover_token,
        "user": pushover_user,
        "message": message
    }
    response = requests.post(pushover_url, data=payload)
    print(response.json())

In [ ]:
push("Hello, world!")

In [ ]:
# Create a tool that the LLM can use to record if a user is interested in knowing more
def record_user_details(email, name="Name not provided", notes="not provided"):
    push(f"User {name} with email {email} is interested in knowing more. Notes: {notes}")
    return {"Recorded": "ok"}

In [ ]:
# Create a tool that the LLM can use to record if it doesn't have the answer
def record_unknown_question(question):
    push(f"recording {question} asked that I couldn't answer")
    return {"Recorded": "ok"}

In [ ]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in knowing more about a topic and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string", "description": "The email address of the user"
            },
            "name": {
                "type": "string", "description": "The name of the user"
            },
            "notes": {
                "type": "string", "description": "Any additional notes about the user"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

In [ ]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Use this tool to record the question that a user asked which you couldn't answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string", 
                "description": "The question that the user asked which you couldn't answer"
            }
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [ ]:
#createa list of tools that are available to the LLM

tools = [
    {
        "type": "function",
        "function": record_user_details_json
    },
    {
        "type": "function",
        "function": record_unknown_question_json
    }
]

In [ ]:
print(tools)

In [ ]:
# This function can take a list of tool calls and run them. 

# def handle_tool_calls(tool_calls):
#     results = []
#     for tool_call in tool_calls:
#         tool_name = tool_call.function.name
#         arguments = json.loads(tool_call.function.arguments)
#         print(f"Tool call: {tool_name}", flush=True)

#         if tool_name == "record_user_details":
#             record_user_details(**arguments)
#         elif tool_name == "record_unknown_question":
#             record_unknown_question(**arguments)
        
#         results.append({
#             "role": "tool", 
#             "content": json.dumps(results),
#             "tool_call_id": tool_call.id
#             }
#         )
#     return results

In [ ]:
# This is a more elegant way that avoids the IF statement.

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [ ]:
# This code will work better if the book is vectorized for ollama. Currently, it uses too many tokens and is unable to answer questions.

# reader = PdfReader("knowledgebase/peng_book.pdf")
# peng_book = ""
# for page in reader.pages:
#     text = page.extract_text()
#     if text:
#         peng_book += text

# with open("knowledgebase/summary.txt", "r", encoding="utf-8") as f:
#     summary = f.read()

# name = "Gautham Pallapa"
# book_title = "Mastering Enterprise Platform Engineering"

# Using the LinkedIn profile as the book needs to be vectorized for ollama to really work well
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

name = "Gautham Pallapar"

In [ ]:
# This is the system prompt for the book landing page

# system_prompt = f"You are acting as {name}. You are answering questions for {name}'s book's landing page, \
# particularly questions related to {name}'s book titled: {book_title}. \
# Your responsibility is to represent {name} for interactions on the book's landing page as faithfully as possible. \
# You are given a summary of {name}'s background and the complete text of {book_title} which you can use to answer questions. \
# Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
# If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
# If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "

# system_prompt += f"\n\n## Summary:\n{summary}\n\n## Book Title:\n{book_title}\n\n## Book Text:\n{peng_book}\n"
# system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

# I am creating a system prompt for the website and the LinkedIn profile
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    print(messages)
    done = False
    
    while not done:
        # Call to the LLM with the tools json
        # If I want to call ollama, I just need to change the client to ollama_client and set the model name to the ollama model name
        response = openai_client.chat.completions.create(
            model=model_name, 
            messages=messages, 
            temperature=0.7, 
            tools=tools
        )
        
        finish_reason = response.choices[0].finish_reason
        
        # If the LLM wants to call a tool, we do that
         
        if finish_reason=="tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(chat, type="messages").launch()